# ThermoBench-Consist Quickstart
CPU-only demo: wrap adapters, run checks for CO₂ and N₂, save a one-page report and two tiny figures.

In [1]:
import json
import os

from thermobench.adapters.adapter_coolprop import CoolPropAdapter
from thermobench.checks import (
    check_clapeyron,
    check_compressibility,
    check_monotonic_rho_isotherm,
)
from thermobench.grid import default_saturation_T, parse_grid_string
from thermobench.report import generate_report
from thermobench.score import aggregate_checks_to_summary

os.makedirs('out', exist_ok=True)

fluid_list = ['CO2', 'N2']
# Per-fluid grids (single-phase sampling band), small/fast:
grids = {
    'CO2': 'T=220:300:20,p=1e5:5e6:5e5',
    'N2' : 'T=80:120:20,p=1e5:3e6:5e5',
}

for fluid in fluid_list:
    grid = grids[fluid]
    T_vals, p_vals = parse_grid_string(grid)
    T0 = float(T_vals[len(T_vals)//2])
    p_line = p_vals
    adapter = CoolPropAdapter(fluid=fluid)

    # C1 & C2 on a single isotherm
    r1 = check_monotonic_rho_isotherm(adapter, fluid, T0, p_line, tol=1e-6)
    r2 = check_compressibility(adapter, fluid, T0, p_line, tol=1e-6)

    # C3 on valid saturation T for the fluid
    Ts = default_saturation_T(fluid)  # CO2 -> [230,240,260,280], N2 -> [85,95,105,115]
    r3 = check_clapeyron(adapter, fluid, Ts, tol_rel=0.1)

    summary = aggregate_checks_to_summary(
        adapter_name=adapter.__class__.__name__, fluid=fluid, grid=grid,
        results_monotonic=[r1], results_compress=[r2], results_clapeyron=[r3],
        tol_monotonic=1e-6, tol_clap=0.1,
    )

    json_path = f'out/report_{fluid}.json'
    with open(json_path, 'w') as f:
        json.dump(summary, f, indent=2)
    md_path = f'out/report_{fluid}.md'
    html_path = f'out/report_{fluid}.html'
    generate_report(summary, md_out=md_path, html_out=html_path, out_dir='out')
    print(f"Saved: {md_path}, {html_path}")

print('Done.')


Saved: out/report_CO2.md, out/report_CO2.html
Saved: out/report_N2.md, out/report_N2.html
Done.
